 ## Step 4: Tối ưu hóa MLPClassifier bằng GridSearchCV

 Mục tiêu: tìm cấu hình tốt nhất cho MLP trên tập dữ liệu Iris.

In [ ]:
print("=== Bước 4.1: Kiểm tra và tạo dữ liệu đầu vào ===")
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

if not os.path.exists("iris_scaled.csv"):
    print("📂 Chưa có file iris_scaled.csv – Đang tạo từ bộ Iris gốc...")
    iris = load_iris()
    data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    data['target'] = iris.target

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data.drop('target', axis=1))
    data_scaled = pd.DataFrame(X_scaled, columns=iris.feature_names)
    data_scaled['target'] = data['target']

    data_scaled.to_csv('iris_scaled.csv', index=False)
    print("✅ Đã lưu iris_scaled.csv (dữ liệu đã chuẩn hóa).")
else:
    print("✅ Đã tìm thấy iris_scaled.csv.")

print("📈 Xem thử 5 dòng đầu:")
print(pd.read_csv("iris_scaled.csv").head())


In [ ]:
print("\n=== Bước 4.2: Cấu hình pipeline và tham số tìm kiếm ===")
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib

df = pd.read_csv("iris_scaled.csv")
X = df.drop("target", axis=1)
y = df["target"]

print(f"🔢 Dữ liệu huấn luyện: {X.shape[0]} mẫu, {X.shape[1]} đặc trưng")

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(max_iter=1000, random_state=42))
])

param_grid = {
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'mlp__activation': ['relu', 'tanh', 'logistic'],
    'mlp__solver': ['adam', 'sgd'],
    'mlp__alpha': [0.0001, 0.001, 0.01],
    'mlp__learning_rate_init': [0.001, 0.01],
}

total_combinations = (
    len(param_grid['mlp__hidden_layer_sizes']) *
    len(param_grid['mlp__activation']) *
    len(param_grid['mlp__solver']) *
    len(param_grid['mlp__alpha']) *
    len(param_grid['mlp__learning_rate_init'])
)
print(f"🔧 Tổng số tổ hợp siêu tham số: {total_combinations}")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
print("\n=== Bước 4.3: Tiến hành huấn luyện bằng GridSearchCV ===")
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

print("⏳ Bắt đầu huấn luyện... (mỗi dấu 'Fitting' là một tổ hợp tham số)")
grid_search.fit(X, y)
print("✅ Huấn luyện hoàn tất.")


In [ ]:
print("\n=== Bước 4.4: Lưu mô hình và xem kết quả ===")
joblib.dump(grid_search, "mlp_gridsearch_model.pkl")
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv("mlp_gridsearch_results.csv", index=False)
print("💾 Đã lưu mô hình và kết quả vào 'mlp_gridsearch_model.pkl' và 'mlp_gridsearch_results.csv'.")

print("\n🎯 Tham số tốt nhất tìm được:")
print(grid_search.best_params_)
print(f"🏆 Độ chính xác cao nhất (cross-validated): {grid_search.best_score_ * 100:.2f}%")

print("\n🔝 Top 3 cấu hình tốt nhất:")
top3 = results_df.sort_values(by='mean_test_score', ascending=False).head(3)
print(top3[['mean_test_score', 'params']])
